In [56]:
import requests
!pip install bs4
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
import pandas as pd
import json
import re

In [57]:
url = 'https://www.livelib.ru/find/books/books'
page = requests.get(url)
page

<Response [200]>

In [58]:
soup = BeautifulSoup(page.text, 'html')
print(soup.prettify())

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">
<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="width=device-width" name="viewport"/>
  <link href="/pushmanifest.json" rel="manifest"/>
  <title id="title-head">
   Результаты поиска: books
  </title>
  <link href="https://www.livelib.ru/find/books/books" rel="canonical"/>
  <link href="https://s.livelib.ru/img/android_phone_icon.png" rel="android-touch-icon"/>
  <link href="/favicon.ico" rel="icon"/>
  <link href="/apple-touch-icon-180x180.png" rel="apple-touch-icon" sizes="180x180"/>
  <link href="https://s.livelib.ru/css/codemirror.202510300623.css" rel="stylesheet" type="text/css"/>
  <link href="https://s.livelib.ru" rel="preconnect"/>
  <link as="style" fetchpriority="high" href="https://s.livelib.ru/css/prod/fonts.202510300623.css" rel="preload"/>
  <link href="https://s.livelib.ru/css/prod/fonts.202510300623.css" media="print" onload

In [59]:
url0 = 'https://www.livelib.ru/book/1012752559-legendy-i-mify-drevnej-gretsii-nikolaj-kun'
page0 = requests.get(url0)
soup0 = BeautifulSoup(page0.text, 'html')
print(soup0.prettify())

<!DOCTYPE HTML>
<html lang="ru">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width,initial-scale=1" name="viewport"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <link href="/pushmanifest.json" rel="manifest"/>
  <title id="title-head">
   Книга Легенды и мифы Древней Греции (сборник) — Николай Кун
  </title>
  <meta content="Читать книгу Легенды и мифы Древней Греции (сборник) ⏩Автор Николай Кун. Год издания 2025. ✍️️Рецензии, цитаты. ⭐Возможность купить" name="description"/>
  <link href="https://www.livelib.ru/book/1012752559-legendy-i-mify-drevnej-gretsii-nikolaj-kun" rel="canonical"/>
  <link href="https://s.livelib.ru" rel="preconnect"/>
  <link href="https://promo.livelib.ru" rel="preconnect"/>
  <link as="image" fetchpriority="high" href="https://s1.livelib.ru/boocover/1012752559/200x305/3677/Nikolaj_Kun__Legendy_i_mify_Drevnej_Gretsii_sbornik.jpg" rel="preload"/>
  <link as="style" fetchpriority="high" href="https://s.livelib.ru

In [ ]:
#<a href="/book/1005584599-vlastelin-kolets-vozvraschenie-korolya-dzhon-r-r-tolkin" title="Джон Р. Р. Толкин - Властелин Колец: Возвращение короля">
unique_urls = set()
for i in tqdm(range(1, 401)): #!!!потом вместо 5 нужно ввести колличество страниц которое будем парсить!!!
    url = 'https://www.livelib.ru/find/books/books/~' + str(i)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html')
    for link in soup.find_all('a'):
        a = link.get('href')
        if a is not None and re.match(r'/book/\d+-.+$', a):
            unique_urls.add('https://www.livelib.ru' + a)
    sleep(1)
# for url in unique_urls:
#     print(url)

  0%|          | 0/400 [00:00<?, ?it/s]

In [ ]:
book_data = []
for url in tqdm(unique_urls, desc="Парсинг книг"):
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')
        # Основная информация о книге
        book_info = {}
        # Название книги
        title_elem = soup.find('h1', class_='bc-header__book-title')
        book_info['title'] = title_elem.text.strip() if title_elem else None
        # Автор
        author_elem = soup.find('h2', class_='bc-header__book-author')
        if author_elem:
            author_link = author_elem.find('a')
            book_info['author'] = author_link.text.strip() if author_link else None
        else:
            book_info['author'] = None
        # Рейтинг
        rating_elem = soup.find('button', class_='bc-rating__btn')
        book_info['rating'] = rating_elem.text.strip() if rating_elem else None
        # Жанры
        genres = []
        genre_elems = soup.find_all('a', class_='bc-genre__link')
        for genre_elem in genre_elems:
            genres.append(genre_elem.text.strip())
        book_info['genres'] = genres
        # Статистика
        stats = {}
        stat_elems = soup.find_all('li', class_='bc-stat__item')
        for stat_elem in stat_elems:
            stat_link = stat_elem.find('a', class_='bc-stat__link')
            if stat_link:
                qty_elem = stat_link.find('span', class_='bc-stat__qty')
                if qty_elem:
                    stat_text = stat_link.text.replace(qty_elem.text, '').strip()
                    stats[stat_text] = qty_elem.text.strip()
        book_info['stats'] = stats
        # Описание книги
        annotation_elem = soup.find('div', class_='bc-about__annotation')
        if annotation_elem:
            txt_elem = annotation_elem.find('p', class_='bc-about__txt')
            book_info['annotation'] = txt_elem.text.strip() if txt_elem else None
        else:
            book_info['annotation'] = None
        # Информация об издании
        edition_info = {}
        edition_elems = soup.find_all('li', class_='bc-edition__item')
        for edition_elem in edition_elems:
            heading_elem = edition_elem.find('span', class_='bc-edition__heading')
            if heading_elem:
                heading = heading_elem.text.strip().replace(':', '')
                links_elems = edition_elem.find_all('a', class_='bc-edition__link')
                if links_elems:
                    edition_info[heading] = [link.text.strip() for link in links_elems]
                else:
                    links_elems = edition_elem.find_all('li', class_='bc-edition__links-item')
                    if links_elems:
                        edition_info[heading] = [item.text.strip() for item in links_elems]

        book_info['edition_info'] = edition_info
        # Дополнительная информация
        additional_info = {}
        info_elems = soup.find_all('li', class_='bc-info__item')
        for info_elem in info_elems:
            title_elem = info_elem.find('summary', class_='bc-info__title')
            if title_elem:
                title = title_elem.text.strip()
                content_elem = info_elem.find('div', class_='bc-info__content')
                if content_elem:
                    txt_elem = content_elem.find('p', class_='bc-info__txt')
                    if txt_elem:
                        additional_info[title] = txt_elem.text.strip()
        book_info['additional_info'] = additional_info
        # ISBN и год издания (извлекаем из скрипта или мета-тегов)
        script_elem = soup.find('script', type='application/ld+json')
        if script_elem:
            try:
                json_data = json.loads(script_elem.string)
                book_info['isbn'] = json_data.get('isbn')
                book_info['publisher'] = json_data.get('publisher', {}).get('name')
                book_info['description'] = json_data.get('description')
            except:
                book_info['isbn'] = None
                book_info['publisher'] = None
                book_info['description'] = None
        book_info['url'] = url
        book_data.append(book_info)
        sleep(0.5)
    except Exception as e:
        print(f"Ошибка при парсинге {url}: {str(e)}")
        continue

In [ ]:
df = pd.DataFrame(book_data)
display(df.head())
display(df.tail())
df.to_csv('litnet_books_dataset.csv', index=False, encoding='utf-8-sig')